In [1]:
import numpy as np
from pathlib import Path
import os
import time
import shutil

#import spikeinterface.full as si
#from spikeinterface.sortingcomponents.motion import correct_motion_on_peaks, interpolate_motion
#from spikeinterface.sorters import run_sorter

from si_utils import *
from SI_H2P import si_h2p

In [2]:
IN1 = 'kendra_scrappy_0125a_g0'
IN2 = 0

spikeglx_folder = f"/ix1/pmayo/lab_NHPdata/{IN1}/"

si_h2p(spikeglx_folder, IMEC=IN2, drift_preset="dredge", plot_figs=True)

Step 1: Setting up folders and reading raw recording...
Step 2: Preprocessing raw recording...
/ix1/pmayo/lab_NHPdata/kendra_scrappy_0125a_g0/figs/preprocess/noise_dists.png


noise_level (workers: 20 processes):   0%|          | 0/20 [00:00<?, ?it/s]

noise_level (workers: 20 processes):   0%|          | 0/20 [00:00<?, ?it/s]

(384,)
bad_channel_ids ['imec0.ap#AP191']


noise_level (workers: 20 processes):   0%|          | 0/20 [00:00<?, ?it/s]

...

Step 4: Saving drift-corrected recording to disk...
write_binary_recording 
engine=process - n_jobs=40 - samples_per_chunk=29,999 - chunk_memory=87.89 MiB - total_memory=3.43 GiB - chunk_duration=1.00s (999.97 ms)


write_binary_recording (workers: 40 processes):   0%|          | 0/3379 [00:00<?, ?it/s]

detect peaks using locally_exclusive (workers: 40 processes):   0%|          | 0/3379 [00:00<?, ?it/s]

localize peaks using center_of_mass (workers: 40 processes):   0%|          | 0/3379 [00:00<?, ?it/s]

PermissionError: [Errno 13] Permission denied: '/rec_output.txt'

In [ ]:
IN1 = 'kendra_scrappy_0124a_g0'
IN2 = 0

save_path = f"/ix1/pmayo/lab_NHPdata/{IN1}/{IN1}_imec{IN2}/dredge/corrected/traces_cached_seg0.raw"
#probe_path = f"/ix1/pmayo/lab_NHPdata/{IN1}/{IN1}_imec{IN2}/{IN1}_t0.imec{IN2}.ap_kilosortChanMap.mat"
probe_path = f"/ix1/pmayo/lab_NHPdata/{IN1}/{IN1}_imec{IN2}/dredge/corrected/probe.prb"

print(save_path)
print(probe_path)

kilosort_h2p(save_path,probe_path,run_ks=True,temp_wh=False)

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
from pathlib import Path
import shutil
import os

import spikeinterface.full as si  # import core only
from spikeinterface.sortingcomponents.peak_detection import detect_peaks
from spikeinterface.sortingcomponents.peak_localization import localize_peaks
from spikeinterface.sortingcomponents.motion import correct_motion_on_peaks

from kilosort import io

#import spikeinterface.extractors as se
#import spikeinterface.preprocessing as spre
#import spikeinterface.sorters as ss
#import spikeinterface.postprocessing as spost
#import spikeinterface.qualitymetrics as sqm
#import spikeinterface.comparison as sc
#import spikeinterface.exporters as sexp
#import spikeinterface.curation as scur
#import spikeinterface.widgets as sw

#from spikeinterface.preprocessing import get_motion_parameters_preset, get_motion_presets
from spikeinterface.sortingcomponents.motion import estimate_motion, interpolate_motion

In [ ]:
############### INPUTS ###############
IN1 = 'kendra_scrappy_0124a_g0'
IN2 = 0

spikeglx_folder = f"/ix1/pmayo/lab_NHPdata/{IN1}/"
preprocess_folder = spikeglx_folder+"figs/preprocess"
motion_folder = spikeglx_folder+f"/{IN1}_imec{IN2}/motion"

os.makedirs(preprocess_folder, exist_ok=True)

#spikeglx_folder = '/ix1/pmayo/lab_NHPdata/kendra_scrappy_0130a_g0'
#dredge_type = 'lfp' # ap

stream_names, stream_ids = si.get_neo_streams('spikeglx', spikeglx_folder)
stream_names

# Read in data

* Neuropixels -- [read_spikeglx](https://spikeinterface.readthedocs.io/en/stable/api.html#spikeinterface.extractors.read_spikeglx)
* Plexon (after converting to binary) -- [read_binary](https://spikeinterface.readthedocs.io/en/stable/api.html#spikeinterface.core.read_binary)

In [ ]:
# Read in raw recording 
RAW_REC = si.read_spikeglx(spikeglx_folder, stream_name=f'imec{IN2}.ap', load_sync_channel=False)
RAW_REC

In [ ]:
# we can estimate the noise on the scaled traces (microV) or on the raw one (which is in our case int16).
noise_levels_microV = si.get_noise_levels(RAW_REC, return_scaled=True)
noise_levels_int16 = si.get_noise_levels(RAW_REC, return_scaled=False)

print(noise_levels_int16.shape)

# Create subplots side by side
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Plot scaled traces (microV)
axes[0].hist(noise_levels_microV, bins=np.arange(5, 30, 2.5))
axes[0].set_title('scaled traces (microV)')
axes[0].set_xlabel('noise [microV]')

# Plot raw traces (int16)
axes[1].hist(noise_levels_int16, bins=np.arange(5, 30, 2.5))
axes[1].set_title('raw traces (int16)')
axes[1].set_xlabel('noise [int16 units]')

output_path = os.path.join(preprocess_folder, 'noise_dists.png')
plt.tight_layout()
plt.savefig(output_path)

# Preprocessing recording
1. Highpass filter
2. Tossing bad channels
3. Common reference

In [ ]:
def preprocess_recording(raw_rec, save_path=None):
    raw_rec = raw_rec.astype('float32')
    
    rec1 = si.highpass_filter(raw_rec, freq_min=400.)
    bad_channel_ids, channel_labels = si.detect_bad_channels(rec1)
    rec2 = rec1.remove_channels(bad_channel_ids)
    print('bad_channel_ids', bad_channel_ids)
    
    rec3 = si.phase_shift(rec2)
    rec4 = si.common_reference(rec3, operator="median", reference="global")
    rec = rec4

    if save_path is not None:
        # here we use static plot using matplotlib backend
        fig, axs = plt.subplots(ncols=3, figsize=(20, 10))
        
        si.plot_traces(rec1, backend='matplotlib',  clim=(-50, 50), ax=axs[0])
        si.plot_traces(rec3, backend='matplotlib',  clim=(-50, 50), ax=axs[1])
        si.plot_traces(rec4, backend='matplotlib',  clim=(-50, 50), ax=axs[2])
        for i, label in enumerate(('hp filter', 'phase shift', 'cmr')):
            axs[i].set_title(label)
    
        
            output_path = os.path.join(save_path, 'preprocess_steps.png')
            plt.tight_layout()
            plt.savefig(output_path)
    
        # plot some channels
        fig, ax = plt.subplots(figsize=(20, 10))
        some_chans = rec.channel_ids[[100, 150, 200, ]]
        si.plot_traces({'filter':rec1, 'cmr': rec4}, backend='matplotlib', mode='line', ax=ax, channel_ids=some_chans)

        output_path = os.path.join(save_path, 'preprocess_chans.png')
        plt.tight_layout()
        plt.savefig(output_path)
    
    return rec, bad_channel_ids

In [ ]:
REC, bad_channel_ids = preprocess_recording(RAW_REC, save_path=None) #preprocess_folder)

In [ ]:
from probeinterface import ProbeGroup, write_prb

probe_path = DATA_DIRECTORY / 'probe.prb'

probe = REC.get_probe()  # From SpikeInterface tutorial, or recording.get_probe()
pg = ProbeGroup()
pg.add_probe(probe)

# CHANGE THIS PATH to wherever you want to save your probe file.
write_prb(probe_path, pg)

print(probe)

In [ ]:
noise_levels_int16 = si.get_noise_levels(REC, return_scaled=False)

job_kwargs = dict(n_jobs=40, chunk_duration='1s', progress_bar=True)
peaks = detect_peaks(REC,  method='locally_exclusive', noise_levels=noise_levels_int16,
                     detect_threshold=5, radius_um=50., **job_kwargs)

peak_locations = localize_peaks(REC, peaks, method='center_of_mass', radius_um=50., **job_kwargs)

In [ ]:
# check for drifts
fs = REC.sampling_frequency
fig, ax = plt.subplots(figsize=(10, 8))
ax.scatter(peaks['sample_index'] / fs, peak_locations['y'], color='k', marker='.',  alpha=0.002)

output_path = os.path.join(preprocess_folder, 'peak_times_depths1.png')
plt.tight_layout()
plt.savefig(output_path)

# Motion/Drift Detection
1. Bandpass filter
2. Common reference
3. Correct motion w/ dredge

In [ ]:
def preprocess_for_drift_correction(raw_rec, bad_channel_ids):
    rec1 = raw_rec.astype('float32')
    
    rec2 = si.bandpass_filter(rec1, freq_min=300.0, freq_max=5000.0)
    rec3 = si.common_reference(rec2, reference="global", operator="median")

    rec4 = rec3.remove_channels(bad_channel_ids)
    print('bad_channel_ids', bad_channel_ids)
    
    return rec4

In [ ]:
FILT_REC1 = preprocess_for_drift_correction(RAW_REC, bad_channel_ids)

## High-level API --> for actually doing drift correction

In [ ]:
DRIFT_REC = si.correct_motion(recording=FILT_REC1, preset="dredge", folder=motion_folder)

In [ ]:
motion_info = si.load_motion_info(motion_folder)
motion = motion_info["motion"]

In [ ]:
fig = plt.figure(figsize=(14, 8))

si.plot_motion_info(motion_info, FILT_REC1, figure=fig, depth_lim=(400, 1000), color_amplitude=True, amplitude_cmap="inferno", scatter_decimate=10,)

output_path = os.path.join(preprocess_folder, 'motion_info.png')
plt.savefig(output_path)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(12, 8), sharey=True)

ax = axs[0]
si.plot_probe_map(FILT_REC1, ax=ax)

peaks = motion_info["peaks"]
sr = FILT_REC1.get_sampling_frequency()
time_lim0 = 750.0
time_lim1 = 1500.0
mask = (peaks["sample_index"] > int(sr * time_lim0)) & (peaks["sample_index"] < int(sr * time_lim1))
sl = slice(None, None, 5)
amps = np.abs(peaks["amplitude"][mask][sl])
amps /= np.quantile(amps, 0.95)
c = plt.get_cmap("inferno")(amps)
axs[0].set_title('Estimated peak locations (pre-dredge)')

color_kargs = dict(alpha=0.2, s=2, c=c)

peak_locations = motion_info["peak_locations"]
# color='black',
ax.scatter(peak_locations["x"][mask][sl], peak_locations["y"][mask][sl], **color_kargs)

peak_locations2 = correct_motion_on_peaks(peaks, peak_locations, motion, FILT_REC1)

ax = axs[1]
si.plot_probe_map(FILT_REC1, ax=ax)
#  color='black',
ax.scatter(peak_locations2["x"][mask][sl], peak_locations2["y"][mask][sl], **color_kargs)
axs[1].set_title('Corrected peak locations (post-dredge)')

ax.set_ylim(400, 600)

output_path = os.path.join(preprocess_folder, 'spatial_spread.png')
plt.savefig(output_path)

# Apply Drift Correction

In [ ]:
#from spikeinterface.sortingcomponents.motion import interpolate_motion

motion_info = si.load_motion_info(motion_folder)
motion = motion_info["motion"]

REC_CORRECTED = interpolate_motion(recording=REC, motion=motion_info['motion'], **motion_info['parameters']['interpolate_motion_kwargs'])

In [ ]:
job_kwargs = dict(n_jobs=40, chunk_duration='1s', progress_bar=True)

DATA_DIRECTORY = Path(spikeglx_folder+f"/{IN1}_imec{IN2}/corrected")

REC_CORRECTED = REC_CORRECTED.save(folder=DATA_DIRECTORY, format='binary', **job_kwargs)

In [ ]:
#from kilosort import io

#DATA_DIRECTORY = Path(spikeglx_folder+f"/{IN1}_imec{IN2}/corrected")
#DATA_DIRECTORY.mkdir(parents=True, exist_ok=True)

#dtype = np.int16
#filename, N, c, s, fs, probe_path = io.spikeinterface_to_binary(
#    REC_CORRECTED, DATA_DIRECTORY, data_name='kendra_scrappy_0130a_g0_t0.imec0_DREDGE.ap.bin', dtype=dtype,
#    chunksize=60000, export_probe=True, probe_name='probe.prb')

# Test run Kilosort

In [ ]:
from kilosort import run_kilosort, DEFAULT_SETTINGS

# NOTE: 'n_chan_bin' is a required setting, and should reflect the total number
#       of channels in the binary file, while probe['n_chans'] should reflect
#       the number of channels that contain ephys data. In many cases these will
#       be the same, but not always. For example, neuropixels data often contains
#       385 channels, where 384 channels are for ephys traces and 1 channel is
#       for some other variable. In that case, you would specify
#       'n_chan_bin': 385.

# Specify probe configuration.
probe = io.load_probe(probe_path)
print(probe)

settings = DEFAULT_SETTINGS
settings['probe'] = probe
settings['n_chan_bin'] = probe['n_chan']+2
settings['fs'] = 30000
settings['data_dir'] = DATA_DIRECTORY.parent
#print(settings)

# This command will both run the spike-sorting analysis and save the results to
# `DATA_DIRECTORY`.
ops, st, clu, tF, Wall, similar_templates, is_ref, \
    est_contam_rate, kept_spikes = run_kilosort(
        settings=settings, probe=probe, data_dtype='int16', save_preprocessed_copy=False
        )